Cài đặt thư viện

In [1]:
# Cài đặt hugging face
!pip install transformer
# Cài đặt thư viện underthesea để thực hiện word segment
!pip install underthesea
# Cài đặt pytorch
!pip install torch
# Cài đặt sklearn
!pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.1 MB/s eta 0:00:00


Load model PhoBERT

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  # Fix potential MKL error
import torch
import numpy as np
import re
import underthesea
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
import json

# Load data from JSON
def load_data_from_json():
    with open('mood_extended.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    texts = []
    labels = []
    for label, sentences in data.items():
        for sentence in sentences:
            texts.append(sentence)
            labels.append(label)
    return texts, labels

# Preprocessing functions (standardize_data, load_stopwords) remain the same


def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer



def standardize_data(row):
    row = re.sub(r"[.,?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ").replace(";", " ").replace("“", " ").replace(":", " ").replace("”", " ").replace('"', " ").replace("'", " ").replace("!", " ").replace("?", " ").replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row



def load_stopwords():
    sw = []
    with open("vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            sw.append(line.replace("\n",""))
    return sw


def make_bert_features(v_text, tokenizer, phobert, sw):  # Pass tokenizer, phobert, and sw as arguments
    v_tokenized = []
    max_len = 100
    for i_text in v_text:
        print("Đang xử lý line = ", i_text)
        line = underthesea.word_tokenize(i_text)
        filtered_sentence = [w for w in line if not w in sw]
        line = " ".join(filtered_sentence)
        line = underthesea.word_tokenize(line, format="text")
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    padded = np.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    attention_mask = np.where(padded == 1, 0, 1)

    padded = torch.tensor(padded).to(torch.long)  # Ensure long type for input_ids
    attention_mask = torch.tensor(attention_mask).to(torch.long)  # Ensure long type for attention_mask

    with torch.no_grad():
        last_hidden_states = phobert(input_ids=padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    return v_features




# Load stopwords and BERT model
print("Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...")
sw = load_stopwords()
print("Đã nạp xong danh sách các từ vô nghĩa")

print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
phobert.eval()  # Put the model in evaluation mode
print("Đã nạp xong model BERT.")

# Load data from JSON
print("Chuẩn bị load dữ liệu....")
text, label_text = load_data_from_json()

# Convert string labels to numerical labels (e.g., "vui" -> 0, "buồn" -> 1)
label_mapping = {label: i for i, label in enumerate(set(label_text))}
label = [label_mapping[l] for l in label_text]

print("Đã load dữ liệu xong")



# Create BERT features
print("Chuẩn bị tạo features từ BERT.....")
features = make_bert_features(text, tokenizer, phobert, sw) # Pass necessary arguments
print("Đã tạo xong features từ BERT")



# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=0)

# Train SVM (using simplified parameter tuning for demonstration)
print("Chuẩn bị train model SVM....")

cl = SVC(kernel='linear', probability=True, gamma='auto')  # Use 'auto' for gamma with linear kernel
cl.fit(X_train, y_train)


sc = cl.score(X_test, y_test)
print('Kết quả train model, độ chính xác = ', sc*100, '%')


# Save the model and label mapping
dump(cl, 'save_model.pkl')
dump(label_mapping, 'label_mapping.pkl')
print("Đã lưu model SVM vào file save_model.pkl và label_mapping.pkl")

Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...
Đã nạp xong danh sách các từ vô nghĩa
Chuẩn bị nạp model BERT....


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Đã nạp xong model BERT.
Chuẩn bị load dữ liệu....
Đã load dữ liệu xong
Chuẩn bị tạo features từ BERT.....
Đang xử lý line =  Tôi đang yêu đời hãy cho tôi một bài hát
Đang xử lý line =  Tôi đang vui hãy cho tôi một bài hát
Đang xử lý line =  Tôi đang hạnh phúc hãy cho tôi một bài hát
Đang xử lý line =  Tôi đang vui vẻ hãy cho tôi một bài hát
Đang xử lý line =  Tui đang cảm thấy hân hoan quá bot ơi.
Đang xử lý line =  Hiện tại tui đang cảm thấy rất tuyệt vời.
Đang xử lý line =  Cảm thấy phấn khởi thì nghe nhạc gì giờ.
Đang xử lý line =  Tôi đang thấy hơi tươi tỉnh, có nhạc gì để nghe không.
Đang xử lý line =  Có bài gì hợp với tâm trạng thoải mái không.
Đang xử lý line =  T đang cảm thấy sung sướng, có bài gì nghe không.
Đang xử lý line =  T đang thấy tự hào, có bài nào để nghe không.
Đang xử lý line =  Gợi ý cho tui một bài hào hứng nhé.
Đang xử lý line =  Gợi ý cho t một bài nồng nhiệt đi.
Đang xử lý line =  Cho tui bài nào rạng rỡ đi.
Đang xử lý line =  Có bài hoan hỉ nào hay hay khôn

In [7]:
# prompt: Sau khi train mô hình ở trên xong và tải xuống thì làm sao để sử dụng mô hình

import os
import torch
import numpy as np
import re
import underthesea
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import load
import json

# Load stopwords and BERT model
def load_stopwords():
    sw = []
    with open("vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            sw.append(line.replace("\n",""))
    return sw

def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

def standardize_data(row):
    row = re.sub(r"[.,?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ").replace(";", " ").replace("“", " ").replace(":", " ").replace("”", " ").replace('"', " ").replace("'", " ").replace("!", " ").replace("?", " ").replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

def make_bert_features(v_text, tokenizer, phobert, sw):
    v_tokenized = []
    max_len = 100
    for i_text in v_text:
        line = underthesea.word_tokenize(i_text)
        filtered_sentence = [w for w in line if not w in sw]
        line = " ".join(filtered_sentence)
        line = underthesea.word_tokenize(line, format="text")
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    padded = np.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    attention_mask = np.where(padded == 1, 0, 1)

    padded = torch.tensor(padded).to(torch.long)
    attention_mask = torch.tensor(attention_mask).to(torch.long)

    with torch.no_grad():
        last_hidden_states = phobert(input_ids=padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    return v_features


# Load the saved model and label mapping
model = load('save_model.pkl')
label_mapping = load('label_mapping.pkl')

# Load stopwords and BERT model
sw = load_stopwords()
phobert, tokenizer = load_bert()
phobert.eval()

# Example usage:
input_text = "Hôm nay tôi rất vui" #Example text
input_text = standardize_data(input_text)
features = make_bert_features([input_text], tokenizer, phobert, sw)
predicted_label_index = model.predict(features)[0]

# Reverse label mapping to get the original label
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_label = reverse_label_mapping[predicted_label_index]

print(f"Predicted label for '{input_text}': {predicted_label}")

Predicted label for 'hôm nay tôi rất vui': buồn


In [6]:
from joblib import load

def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

# Load the saved model and label mapping
model = load('save_model.pkl')
label_mapping = load('label_mapping.pkl')

# Inverted label mapping for getting string labels from predictions
inverted_label_mapping = {v: k for k, v in label_mapping.items()}
# Load PhoBERT and tokenizer (only once)
phobert, tokenizer = load_bert()
phobert.eval()  # Put in evaluation mode
sw = load_stopwords()


def predict_sentiment(text):
    """Predicts the sentiment of a given text."""

    # Preprocess the text
    processed_text = standardize_data(text)

    # Create BERT features
    features = make_bert_features([processed_text], tokenizer, phobert, sw)

    # Make prediction
    prediction = model.predict(features)[0]  # Get the predicted class (integer)

    # Get the string label
    sentiment = inverted_label_mapping[prediction]

    return sentiment



# Example usage:
text = "Tôi đang cảm thấy rất vui"
sentiment = predict_sentiment(text)
print(f"Sentiment: {sentiment}")

text = "Bài hát này thật buồn"
sentiment = predict_sentiment(text)
print(f"Sentiment: {sentiment}")


# Example with multiple sentences
sentences = [
    "Hôm nay trời đẹp quá",
    "Bây giờ tôi đang cảm thấy tuyệt vời, bạn nghĩ tôi nên nghe nhạc gì?",
    "Kết quả thật đáng thất vọng",
    "Tôi đang rất buồn và cô đơn",
]

for sentence in sentences:
    sentiment = predict_sentiment(sentence)
    print(f"Sentence: {sentence}, Sentiment: {sentiment}")

Đang xử lý line =  tôi đang cảm thấy rất vui
Sentiment: vui
Đang xử lý line =  bài hát này thật buồn
Sentiment: buồn
Đang xử lý line =  hôm nay trời đẹp quá
Sentence: Hôm nay trời đẹp quá, Sentiment: vui
Đang xử lý line =  bây giờ tôi đang cảm thấy tuyệt vời  bạn nghĩ tôi nên nghe nhạc gì
Sentence: Bây giờ tôi đang cảm thấy tuyệt vời, bạn nghĩ tôi nên nghe nhạc gì?, Sentiment: vui
Đang xử lý line =  kết quả thật đáng thất vọng
Sentence: Kết quả thật đáng thất vọng, Sentiment: buồn
Đang xử lý line =  tôi đang rất buồn và cô đơn
Sentence: Tôi đang rất buồn và cô đơn, Sentiment: buồn
